1. 문서의 내용을 읽는다
2. 문서를 쪼갠다
 - 토큰 수 초과로 답변을 생성하지 못할 수 있고
 - 문서가 길면 (인풋이 길면) 답변 생성이 오래걸림
3. 임베딩 -> 벡터 데이터베이스에 저장
4. 질문이 있을 때, 백터 데이터베이스에 유사도 검색
5. 유사도 검색으로 가져온 문서를 LLM에 질문과 같이 전달

In [1]:
%pip install --upgrade --quiet  docx2txt langchain-community
%pip install -qU langchain-text-splitters
%pip install langchain-chroma
%pip install -U langchain langchainhub --quiet

Note: you may need to restart the kernel to use updated packages.


  DEPRECATION: docx2txt is being installed using the legacy 'setup.py install' method, because it does not have a 'pyproject.toml' and the 'wheel' package is not installed. pip 23.1 will enforce this behaviour change. A possible replacement is to enable the '--use-pep517' option. Discussion can be found at https://github.com/pypa/pip/issues/8559

[notice] A new release of pip available: 22.3.1 -> 24.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [55]:
# 1. 문서의 내용을 읽는다
# 2. 문서를 쪼갠다
from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,
    chunk_overlap=200,
)

loader = Docx2txtLoader('./tax.docx')
document_list = loader.load_and_split(text_splitter=text_splitter)

In [58]:
# 3. 임베딩 -> 벡터 데이터베이스에 저장
from dotenv import load_dotenv
from langchain_openai import OpenAIEmbeddings

load_dotenv()

embedding = OpenAIEmbeddings(model='text-embedding-3-large')


In [65]:
from langchain_chroma import Chroma

# database = Chroma.from_documents(documents=document_list, embedding=embedding, collection_name="chroma-tax", persist_directory="./chroma")
database = Chroma(collection_name="chroma-tax", persist_directory="./chroma", embedding_function=embedding)

# 4.유사도 검색

In [60]:
query = '연봉 5천만원인 직장인의 소득세는 얼마인가요?'
# retrieved_docs = database.similarity_search(query, k=3)

In [61]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model='gpt-4o')

In [62]:
from langchain import hub

prompt = hub.pull("rlm/rag-prompt")

# 5. LLM 질의

In [66]:
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=database.as_retriever(),
    chain_type_kwargs={"prompt": prompt}
)

In [67]:
ai_message = qa_chain({"query": query})

In [68]:
ai_message

{'query': '연봉 5천만원인 직장인의 소득세는 얼마인가요?',
 'result': '연봉 5천만원인 직장인의 소득세는 기본세율에 따릅니다. 소득세 기본세율은 과세표준에 따라 결정되며, 구체적인 세율은 세법에 따라 달라질 수 있습니다. 따라서, 정확한 소득세 금액을 계산하려면 세율표를 참조해야 합니다.'}